# Пара 2

Доучим основы (базу) и на следующей паре уже начнем наводить суету

## Словари

Очень удобные способ хранения любых хоть сколько-то организованных данных. По факту - полностью перенесенный в питон и немного дополненный формат неряционных бд .json

В чем премис: у нас какие-то данный формата имя, фамилия, рост, кличка собаки и тп... Что угодно, что соответсвует шаблону `key -> value`. Мы можем сопаставить каким-то общим названиям конкретные значения для каждого случая (Имя -> Tape, Фамилия -> BigBaby) 

In [10]:
d = {"price": 300, "currency": "bucks", "three_hundred": 300, "name": "Van"}
print(d["price"], d["currency"])
print(d["three_hundred"]/3, '\n')

#по дефолту итератор возвращает массив ключей
for key in d:
    print("массив - тоже итерируемый объект:", d[key])
    
print("")
    
for key, value in d.items():
    if type(value) == int:
        print(key, " - целочисленное значение")

300 bucks
100.0 

массив - тоже итерируемый объект: 300
массив - тоже итерируемый объект: bucks
массив - тоже итерируемый объект: 300
массив - тоже итерируемый объект: Van

price  - целочисленное значение
three_hundred  - целочисленное значение


Словари еще очень удобны тем, что в `value` можно положить __любой__ объект, в том числе и другой словарь. Из этого следует, что из словарей можно построить произвольное дерево. Если вы хотя бы немного интересовались структурами данных, вы понимаете почему это прекрасно ( ͡° ͜ʖ ͡°) 

In [15]:
petwalk = {"Clara": [{"Name": "Ruby", 
                     "Animal": "Dog", 
                     "Age": 4, 
                     "Breed": "Golden Retriever"}], 
           "Mark": [{"Name": "Victor",
                    "Animal": "Dog",
                    "Age": 7, 
                    "Breed": "Irish Setter"}, {"Name": "Hmyr", "Animal": "Ferret", "Age": 1}]
          }

a = petwalk["Clara"][0]["Age"]
print("Ruby's age is", a)

for q in petwalk:
    print(q, "'s pets are", *[i["Name"] for i in petwalk[q]])

Ruby's age is 4
Clara 's pets are Ruby
Mark 's pets are Victor Hmyr


А теперь разберем ранее обещанный `**kwargs`. Очень похож на `*args`, но распаковывает в функцию не массив, а словарь. Позволяет обобщить всяческую "околофлаговую" семантику и опциональные аргументы, написав вместо 500 значений одно слово

In [30]:
cfg_1 = {"round": False, "splitter": ",", "max_len": 5}
cfg_2 = {"round": "ceil", "max_len": 7}

def hell_sum(*args, **kwargs):
    ret_val = sum(args)
    if kwargs["round"]:
        if kwargs["round"] == "floor":
            ret_val = int(ret_val)
        elif kwargs["round"] == "ceil":
            ret_val = int(ret_val)+1
    else: 
        ret_val = str(ret_val)
        ret_val.replace(".", kwargs["splitter"])
    ret_val = str(ret_val)
    edge = kwargs["max_len"]+1 
    return ret_val[:edge]

print("cfg1:", hell_sum(12.3421, 3, 15.15161, 5.3, **cfg_1))
print("cfg2.1:", hell_sum(12.3421, 3, 15.15161, 5.3, **cfg_2))
print("cfg2.2:", hell_sum(123421, 3, 1515161, 5.3, **cfg_2))

cfg1: 35.793
cfg2.1: 36
cfg2.2: 1638591


## Начала ООП

Объекты по своей сути продолжают тот принцип обобщения, по которому работают циклы и функции на более высокий уровень абстрактции. Одна из важных киллер-фич - возможность взаимодействовать с outer-scope'ом из функциии делать это сравнительно безопасно. А еще у объектов есть свой словарь анимешника - переменные здесь называются __полями__, а функции - __методами__

Объекты нужны, чтобы описывать произвольные множества и как-то определять операции над ними. Встроенная система типов любого языка построена максимально универсальной, но возможность создавать какие-либо свои типы данных бывает нередко очень полезна и сильно сокращает время написания кода. ООП - один из ~~костылей~~ возможных методов реализации этой потребности

Давайте попробуем реализовать какое-нибудь математическое множество со сложной структурой с которым вы при этом уже не раз сталкивались в математике. В качестве оного выберем множество многочленов

Сам в себе многочлен можно описать довольно несложно - это просто вектор произвольной длины, где каждый его элемент - коэффициент со степенью, соответствующей его номеру

In [31]:
class Polynomial:
    def __init__(self, *args):
        self.coefs = args[::-1]

Класс объявляется, как ни странно, словом `class`. После этого обычно сразу объявляются поля в формате `self.field_name = "value"`. `self` перед полями нужен для того. чтобы обозначить питону, чтобы мы хотим иметь к ним доступ из любого метода класса, в котором тоже написано `self` в качестве аргумента. Страшно выглядящий заключенный в два подчеркивания метод `__init__` описывает поведение при создании __экземпляра__ нашего класса, то есть какой-то конкретной сущности из нашего абстрактного множества, если переводить с математического. `__init__` - первый из так называемых "магических методов", которые обычно делают всякое странное-полезное. 

Мы разворачиваем полученный при инициализации массив в обратном порядке, потому что обычно люди записывают коэффициенты многочленов начиная со старшей степени, а вот компьютеру проще с ними обращаться в обратном порядке

In [45]:
class Polynomial:
    def __init__(self, *args):
        self.coefs = list(args[::-1])
        
    def at(self, point):
        ret_val = 0
        for degree, coef in enumerate(self.coefs):
            ret_val += coef*point**degree
        return ret_val
    
#создаем многочлен 2x^2-x+3
poly1 = Polynomial(2, -1, 3)

#считаем многочлен в двух точках
print("poly1 at 0 equals:", a.at(0))
print("poly1 at 3 equals:", a.at(3))

poly1 at 0 equals: 3
poly1 at 3 equals: 18


Давайте также научимся складывать наши многочлены, причем складывать обычным символом `+`. Для этого нам потребуется еще один магический метод - `__add__`. Также добавим метод вывода `show` для того, чтобы видеть, как выглядел бы наш многочлен на бумаге

In [76]:
class Polynomial:
    def __init__(self, *args):
        self.coefs = list(args[::-1])
    
    def at(self, point):
        ret_val = 0
        for degree, coef in enumerate(self.coefs):
            ret_val += coef*point**degree
        return ret_val
    
    def show(self):
        ret_val = ""
        for degree, coef in enumerate(self.coefs):
            if coef != 0:
                if degree != 0:
                    ret_val += "{}x^{} + ".format(coef, degree)
                else:
                    ret_val += "{} + ".format(coef)
        ret_val = ret_val[:-3]
        print(ret_val)
    
    def __add__(self, other):
        if len(self.coefs) >= len(other.coefs):
            res = [0 for i in self.coefs]
            min_l = len(other.coefs)
            max_l = len(self.coefs)
            for i in range(min_l):
                res[i] = self.coefs[i] + other.coefs[i]
            for i in range(min_l, max_l):
                res[i] = self.coefs[i]
        else:
            res = [0 for i in other.coefs]
            min_l = len(self.coefs)
            max_l = len(other.coefs)
            for i in range(min_l):
                res[i] = self.coefs[i] + other.coefs[i]
            for i in range(min_l, max_l):
                res[i] = other.coefs[i]
        
        
        return Polynomial(*res[::-1])
    
A = Polynomial(1, -2, 3, 4)
B = Polynomial(3, 0, 1)
A.show()
B.show()
C = B + A
C.show()

4 + 3x^1 + -2x^2 + 1x^3
1 + 3x^2
5 + 3x^1 + 1x^2 + 1x^3
